In [2]:
from os.path import join
import keras
from keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, Callback
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import Adam

import tensorflow as tf
import pandas as pd
import os
import numpy as np
import csv
import sys
sys.path.append("../")

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

import gzip, pickle

## VGG16

In [3]:
model_vgg16 = load_model('../models/VGG16-finetuned-1_epochs.h5')

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
for i, layer in enumerate(model_vgg16.layers):
    model_vgg16.layers[i].trainable = False
    model_vgg16.layers[i].name = '{}_{}'.format(layer.name, 'vgg16')

In [6]:
vgg16_out = model_vgg16.output

## VGG19

In [10]:
model_vgg19 = load_model('../models/VGG19-fc-1_epochs.h5')

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [11]:
for i, layer in enumerate(model_vgg19.layers):
    model_vgg19.layers[i].trainable = False
    model_vgg19.layers[i].name = '{}_{}'.format(layer.name, 'vgg19')

In [12]:
vgg19_out = model_vgg19.output

## ResNet50

In [7]:
model_resnet50 = load_model('../models/ResNet50-finetuned-1_epochs.h5')

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
for i, layer in enumerate(model_resnet50.layers):
    model_resnet50.layers[i].trainable = False
    model_resnet50.layers[i].name = '{}_{}'.format(layer.name, 'resnet50')

In [9]:
resnet50_out = model_resnet50.output

# Concatenation

In [14]:
merge_0 = keras.layers.concatenate([vgg16_out, vgg19_out, resnet50_out])

In [15]:
output = Dense(228, activation='sigmoid', name='main_output')(merge_0)

In [17]:
model = Model(inputs=[model_vgg16.input, model_vgg19.input, model_resnet50.input], outputs=output)

# Testing

In [23]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [22]:
# Images
images_path_train = os.path.abspath('../data/train/')
images_path_validation = os.path.abspath('../data/validation/')
images_path_test = os.path.abspath('../data/test/')

# Labels
with gzip.open('../data/y_train.pickle','rb') as fp:
    y_train = pickle.load(fp)
with gzip.open('../data/y_validation.pickle','rb') as fp:
    y_validation = pickle.load(fp)

In [24]:
batch_size=128*2 # 128 per GPU
#epochs = 10
#batch_size = 64

# load the generators
training_gen = BatchGenerator(input_dir=images_path_train, y=y_train, batch_size=batch_size)

In [31]:
validation_gen = BatchSequence(input_dir=images_path_validation, y=y_validation, batch_size=batch_size)

In [37]:
# Multi-GPU data parallelism
multi_model = multi_gpu_model(model, gpus=4)

In [41]:
# Use binary loss instead of categorical loss to penalize each output independently, also use lower learning rate
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
multi_model.compile(optimizer=optimizer, loss='binary_crossentropy')

In [45]:
modelname = 'ensemble'
epochs = 1
steps = int(y_train.shape[0]/batch_size) + 1

In [50]:
for i in range(epochs):
    history = multi_model.fit_generator([training_gen, training_gen, training_gen], steps_per_epoch=steps, epochs=1)
    model.save("models/{}-model-{}_epochs.h5".format(modelname, i+1))

Epoch 1/1


TypeError: 'list' object is not an iterator

In [36]:
def train_network(modelname, epochs):    
    with tf.device('/cpu:0'):
        model = load_model('models/{}-fc-1_epochs.h5'.format(modelname))

        trainable_layers = layer_dict[modelname[:5]]
        for layer in model.layers[:trainable_layers]:
            layer.trainable = False
        for layer in model.layers[trainable_layers:]:
            layer.trainable = True

    # Multi-GPU data parallelism
    multi_model = multi_gpu_model(model, gpus=4)

    # Use binary loss instead of categorical loss to penalize each output independently, also use lower learning rate
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    multi_model.compile(optimizer=optimizer, loss='binary_crossentropy')

    steps = int(y_train.shape[0]/batch_size) + 1

    #history = model.fit_generator(training_gen, steps_per_epoch=steps, epochs=1, callbacks=[checkpoint, metrics], validation_data=val_gen)
    for i in range(epochs):
        history = multi_model.fit_generator(training_gen, steps_per_epoch=steps, epochs=1)
        model.save("models/{}-finetuned-{}_epochs.h5".format(modelname, i+1))

In [ ]:
model.save('./ensemble_model_1000.h5')